In [3]:
#!pip install pyspark

In [4]:
#!pip install setuptools

In [5]:
# se usiamo master/worker locali non usare

# import findspark
# findspark.init()

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Recommendations with ALS (Big Dataset) ') \
    .master('yarn') \
    .config('spark.driver.memory', '3g') \
    .config('spark.executor.instances', '3') \
    .config('spark.executor.memory', '6g') \
    .config("spark.locality.wait.node", "0") \
    .getOrCreate()
    #.config("spark.hadoop.fs.defaultFS", "hdfs://localhost:50054") \

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/10 12:26:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/10 12:26:37 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [8]:
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.4


In [9]:
spark.sparkContext._conf.getAll()  # check the config

[('spark.app.name', 'Recommendations with ALS (Big Dataset) '),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.executor.instances', '3'),
 ('

In [3]:
hdfs_port = 9000

In [14]:
import pandas as pd
import numpy as np

#ratings = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/ratings.csv', inferSchema=True, header=True)
ratings = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/ratingsBig.csv', header=True, inferSchema=True)
ratings.limit(5).toPandas()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [15]:
#movies = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/movies.csv', inferSchema=True, header=True)
movies = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/moviesBig.csv', header=True, inferSchema=True)
movies.limit(5).toPandas()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
user_movie_ratings = ratings.join(movies, on='movieId', how='inner')
user_movie_ratings.limit(5).toPandas()

,movieId,userId,rating,timestamp,title,genres
0,1,1,4.0,1225734739,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,110,1,4.0,1225865086,Braveheart (1995),Action|Drama|War
2,158,1,4.0,1225733503,Casper (1995),Adventure|Children
3,260,1,4.5,1225735204,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
4,356,1,5.0,1225735119,Forrest Gump (1994),Comedy|Drama|Romance|War


In [14]:
user_movie_ratings.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [17]:
# Optimizing the data
ratings = 1
# From the ALS documentation, genres and timestamps do not have an impact on the recommendations
user_movie_ratings = user_movie_ratings.drop('genres')
user_movie_ratings = user_movie_ratings.drop('timestamp')

In [18]:
# Check the memory occupation
size = user_movie_ratings.rdd.map(lambda x: len(str(x))).sum()

print(f"Estimated memory usage: {size/(1024**2)} MB")

[Stage 81:=================================================>        (6 + 1) / 7]

Estimated memory usage: 2537.9408855438232 MB


In [12]:
import time

test_user = 1
user_movie_ratings.filter(user_movie_ratings.userId == test_user).show()
start_time=time.time()
print(user_movie_ratings.filter(user_movie_ratings.userId == test_user).count())
print("--- %s seconds ---" % (time.time() - start_time))

+-------+------+------+--------------------+
|movieId|userId|rating|               title|
+-------+------+------+--------------------+
|      1|     1|   4.0|    Toy Story (1995)|
|      3|     1|   4.0|Grumpier Old Men ...|
|      6|     1|   4.0|         Heat (1995)|
|     47|     1|   5.0|Seven (a.k.a. Se7...|
|     50|     1|   5.0|Usual Suspects, T...|
|     70|     1|   3.0|From Dusk Till Da...|
|    101|     1|   5.0|Bottle Rocket (1996)|
|    110|     1|   4.0|   Braveheart (1995)|
|    151|     1|   5.0|      Rob Roy (1995)|
|    157|     1|   5.0|Canadian Bacon (1...|
|    163|     1|   5.0|    Desperado (1995)|
|    216|     1|   5.0|Billy Madison (1995)|
|    223|     1|   3.0|       Clerks (1994)|
|    231|     1|   5.0|Dumb & Dumber (Du...|
|    235|     1|   4.0|      Ed Wood (1994)|
|    260|     1|   5.0|Star Wars: Episod...|
|    296|     1|   3.0| Pulp Fiction (1994)|
|    316|     1|   3.0|     Stargate (1994)|
|    333|     1|   5.0|    Tommy Boy (1995)|
|    349| 

In [9]:
train, test = user_movie_ratings.randomSplit([0.8, 0.2], seed=40)

In [10]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=10,
          regParam=0.1,
          rank=20,
          userCol='userId', 
          itemCol='movieId', 
          ratingCol='rating', 
          nonnegative=True,
          coldStartStrategy='drop')

"""
als = ALS(userCol='userId', 
          itemCol='movieId', 
          ratingCol='rating', 
          nonnegative=True,
          coldStartStrategy='drop')
"""

"\nals = ALS(userCol='userId', \n          itemCol='movieId', \n          ratingCol='rating', \n          nonnegative=True,\n          coldStartStrategy='drop')\n"

In [15]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[als])

In [16]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [15, 20, 25]) \
    .addGrid(als.regParam, [0.01, 0.1, 1]) \
    .addGrid(als.maxIter, [10]) \
    .build()

In [17]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse',
                                predictionCol='prediction',
                                labelCol='rating')

In [18]:
cross_validator = CrossValidator(estimator=pipeline,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=3)

In [13]:
start_time = time.time()

model = als.fit(train)
#cv_model = cross_validator.fit(train)
#model = cv_model.bestModel

print("--- %s seconds ---" % (time.time() - start_time))

--- 15.558064937591553 seconds ---


In [23]:
print(model.stages[0].rank) 
print(model.stages[0]._java_obj.parent().getMaxIter()) 
print(model.stages[0]._java_obj.parent().getRegParam())

20
10
0.1


In [25]:
start_time = time.time()
predicted_ratings = model.transform(test)
print("--- %s seconds ---" % (time.time() - start_time))
predicted_ratings.limit(5).toPandas()

--- 0.05255317687988281 seconds ---


,movieId,userId,rating,timestamp,title,genres,prediction
0,1,31,5.0,850466616,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.301744
1,1,33,3.0,939647444,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.885135
2,1,54,3.0,830247330,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.118542
3,1,57,5.0,965796031,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.493537
4,1,63,5.0,1443199669,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.625635


In [26]:
rmse = evaluator.evaluate(predicted_ratings)
print(rmse)

0.8740833245044529


In [29]:
def test_ALS(user, n):
    user_movies = user_movie_ratings.filter(user_movie_ratings.userId == user)
    print(f'User {user} has rated {user_movies.select("movieId").distinct().count()} movies')
    recommendations = model.transform(user_movies)
    recommendations = recommendations.sort("prediction", ascending=False)
    recommendations = recommendations.join(movies, on="movieId", how='inner')
    return recommendations.limit(n).toPandas()

In [30]:
user = 1
n = 10
test_ALS(user, n)

User 1 has rated 232 movies


,movieId,userId,rating,timestamp,title,genres,prediction,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.200477,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3,1,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,3.983679,Grumpier Old Men (1995),Comedy|Romance
2,6,1,4.0,964982224,Heat (1995),Action|Crime|Thriller,4.421932,Heat (1995),Action|Crime|Thriller
3,47,1,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.706397,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,50,1,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.690398,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,70,1,3.0,964982400,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,3.643147,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
6,101,1,5.0,964980868,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance,4.854663,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance
7,110,1,4.0,964982176,Braveheart (1995),Action|Drama|War,4.609159,Braveheart (1995),Action|Drama|War
8,151,1,5.0,964984041,Rob Roy (1995),Action|Drama|Romance|War,3.731902,Rob Roy (1995),Action|Drama|Romance|War
9,157,1,5.0,964984100,Canadian Bacon (1995),Comedy|War,4.143459,Canadian Bacon (1995),Comedy|War


In [31]:
from pyspark.sql.functions import lit

# Recommend to the user movies he has not rated yet
def recommend(user, n):
    user_movies = user_movie_ratings.filter(user_movie_ratings.userId == user)
    user_rated_movies = user_movies.select('movieId').distinct()
    all_movies = user_movie_ratings.select('movieId').distinct()
    movies_to_recommend = all_movies.subtract(user_rated_movies)
    
    user_to_recommend = movies_to_recommend.withColumn('userId', lit(user))
    print(f'User {user} has not seen {user_to_recommend.select("movieId").distinct().count()} movies')

    recommendations = model.transform(user_to_recommend)
    recommendations = recommendations.join(movies, on='movieId', how='inner')

    recommendations = recommendations.sort('prediction', ascending=False)
    
    return recommendations.limit(n)

In [32]:
recs = recommend(user, n)
recs.toPandas()

User 1 has not seen 9492 movies


,movieId,userId,prediction,title,genres
0,68945,1,5.648950,Neon Genesis Evangelion: Death & Rebirth (Shin...,Action|Animation|Mystery|Sci-Fi
1,3379,1,5.648950,On the Beach (1959),Drama
2,171495,1,5.563516,Cosmos,(no genres listed)
3,158966,1,5.418531,Captain Fantastic (2016),Drama
4,141718,1,5.417383,Deathgasm (2015),Comedy|Horror
5,78836,1,5.409784,Enter the Void (2009),Drama
6,8477,1,5.387444,"Jetée, La (1962)",Romance|Sci-Fi
7,26928,1,5.375473,"Summer's Tale, A (Conte d'été) (1996)",Comedy|Drama|Romance
8,26073,1,5.375473,"Human Condition III, The (Ningen no joken III)...",Drama|War
9,74226,1,5.375473,"Dream of Light (a.k.a. Quince Tree Sun, The) (...",Documentary|Drama


In [31]:
# Connect to Neo4j
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "testtest"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [29]:
upload_test=True

In [ ]:
# Delete old test
if upload_test:
    def delete_test(tx, userId, movieId):
        tx.run("MATCH (u:User {userId: $userId})-[r:ALS_RECOMMENDED]->(m:Movie {movieId: $movieId}) DELETE r", userId=userId, movieId=movieId)

    with driver.session() as session:
        for movieId in recs:           
            session.execute_write(delete_test, user, movieId['title'])

In [ ]:
# Upload the test
if upload_test:
    def create_recommendations(tx, userId, recs):
            for rec in recs.collect():
                tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
                    "MERGE (u)-[:ALS_RECOMMENDED]->(m)",
                    userId=userId, title=rec['title'])

    with driver.session() as session:
        session.execute_write(create_recommendations, user, recs)

In [43]:
upload_predictions = True

In [ ]:
# Predict all users
def create_recommendations(tx, userId, recs):
    for rec in recs.collect():
        tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
               "MERGE (u)-[:ALS_RECOMMENDED]->(m)",
               userId=userId, title= rec['title'])
                
if upload_predictions:
    with driver.session() as session:
        start_time = time.time()
        for user in user_movie_ratings.select('userId').distinct().collect():
            recs = recommend(user['userId'], 10)
            session.execute_write(create_recommendations, user['userId'], recs)
        print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
driver.close()

In [26]:
spark.stop()